# Parsing

## Parsing Basics

- Parsing or syntax anaylsis has two goals
 - To find and report all syntax errors in the code
 - To generate the syntactic structure of the code
- We can parse any unambigous grammar in O($n^3$)
 - By placing more restrictions on the form of the grammar, this can be reduced to O($n$)
- There are two general parsing strategies
 - Top Down, Start with the root and go to the leaves
 - Bottom Up, Start with the leaves, work towards the root

## Restricting the Grammar
- We define two classes of grammar that garuantee linear (O(n)) parsing time.
- LL grammars
    - Symbols are read left to right, results in a leftmost derivation
    - Parsed by LL parsers
- LR grammars
    - Symbols are read left to right, results in a rightmost derivation 
    - Parsed by LR parsers

## Top-Down vs Bottom-Up
- Top down is 
    - Easy to Implement
    - Requires more preprocessing of the grammar
- Bottom up is 
    - Harder to implement by hand
        - Not hard to generate however
    - Can work with most grammars as is

## Top-Down Parsing
- Corresponds to a left-most derivation
- Every node is visited before visiting its branches
- Two common implementations are __recursive-descent parsing__ and __table-driven parsing__
 - Both of these are __LL__ algorithms, using a left-to-right scan on the input and a leftmost derivation

## Resursive-Descent Parsing

- Most popular Top-Down Parsing method
- Is a collection of subprograms (functions)
 - Each non-terminal in a grammar has exactly one subprogram associated with it
 - We assume a global *next_token* that holds next token in the input
 - We also assume that each subprogram leaves the global *next_token* on the token after it.

## Recursive-Descent Parsing Example

For the grammar 

$< term > \to < factor > \{(*|/)< factor >\}*$  


We could use the following recursive descent parsing subprogram (this one is written in C)

```c
 void term() {
     factor(); /* parse first factor*/
     while (next_token == ast_code ||
            next_token == slash_code) {
         lexical(); /* get next token */
         factor(); /* parse next factor */
     }
 }
```

## Recursive Descent Parsing Trace
- Using the handout, find the trace for the following strings:
        - (sum + 47) / total (As a class)
        - a * (b + c)

## Recursive-Descent Problems

- Cannot handle Grammars of the form 
 - $< expr > \to < expr > + < term >$
 - Will always call the function __expr__, whose first call is to __expr__  
 
- Will have to do lots of backtracking for non-terminals that have many potential rules
 - <span style="font-size:.7em">$< expr > \to < var > | < var > + < var > | < var > - < var > | < var > * < var> $</span>
- We can fix both these issues by rewritting the grammar

## Removing Left Recursion

For a grammar of

<ul style="list-style-type:none;">
<li>$ S \to S \; \alpha $ </li>
<li>$ S \to \beta $ </li>
<li>$ \alpha $ , $ \beta$ are a mixture of terminals and non-terminals </li>
</ul>

We can remove left recursion by replacing S with 

<ul style="list-style-type:none;">
<li>$ S \to \beta \; S' $ </li>
<li>$ S' \to \alpha \; S' \, |  \, \epsilon $ </li>
</ul>

## Removing Left Recursion Practice

Remove left recursion from 

- $ E \to E \, + \, T \, | \, T$
- $ T \to T \, * \, F \, | \, F$



## Predictive Parsing
- As defined, top-down parsers will often have to backtrack
    - This reduces their efficency 
- By peeking ahead in the input $k$ symbols, we can guess which rule to use, and are less likely to backtrack
    - We denote a grammar that can be parsed this way as an LL(k) grammar or an LR(k) grammar
    - LL(1) grammars are usually sufficient to make a top-down parser efficient 
    

## Left Factoring
 - The other issue for Recursive-Descent parsers are statements such as 
  - V $\to$ I | I [ E ]
  - In both cases, the next token would be I, so our parser would have to guess
 - This can be rewritten as 
  - V $\to$ I N
  - N $\to \epsilon$ | [ E ] 

## Parsing Tables
- A grammar that is in LL(1) form has only one production for each non-terminal and input symbol
- We create a sparse table, with
    - The rows as the current non-terminal
    - The columns as the next input symbol
    - The cell value as the next action, or empty if an error
- Need a stack to keep track of pending non-terminals

## Parsing Table Algorithm
- Initialize stack as &lt;START_SYMBOL&gt; and next
- While stack is not empty
    - If STACK[0] is a Non-Terminal Symbol
        - Pop `STACK`
        - Push `Parse_Table[STACK[0],next]` onto `STACK`
    - If `STACK[0]` is a Terminal Symbol
        - Pop `STACK`

 ## Parsing Tables Practice
 <div stlye="width:100%">

 <div style="float:left;width:40%;margin:0px auto">
<ul>
<li>Given the following grammar:
<ul>
<li>E $\to$ TX</li>
<li>X $\to$ + E | $\epsilon $</li>
<li>T $\to$ int Y | ( E ) </li>
<li>Y $\to$ $*$ T | $\epsilon$</li>
</ul>
</li>
<li>What is the parse of int $*$ int? (As a Class)</li>
<li>What is the parse of int $+$ int?</li>
</ul>
     
 </div>
 <div style="float:right;width:60%;margin:0px auto">
Given the following parse table
 <table style="margin:0px auto;font-size:1.25em;text-align:center">
 <thead>
 <tr>
 <th></th>
 <th>int</th>
 <th> $*$ </th>
 <th>$+$</th>
 <th style="text-align:center">(</th>
 <th style="text-align:center">)</th>
 <th style="text-align:center">\$</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>E</td>
 <td>T X</td>
 <td></td>
 <td></td>
 <td>T X</td>
 <td></td>
 <td></td>
 </tr>
  <tr>
 <td>X</td>
 <td></td>
 <td></td>
 <td>+ E</td>
 <td></td>
 <td>$\epsilon$</td>
 <td>$\epsilon$</td>
 </tr>
   <tr>
 <td>T</td>
 <td>int Y</td>
 <td></td>
 <td></td>
 <td>( E )</td>
 <td></td>
 <td></td>
 </tr>
    <tr>
 <td>Y</td>
 <td></td>
 <td>* T</td>
 <td>$\epsilon$</td>
 <td></td>
 <td>$\epsilon$</td>
 <td>$\epsilon$</td>
 </tr>
 </tbody>
 </table>
 </div>
 </div>